# Rainfall Erosivity Estimation

## Storm Idenfication

In [ ]:
import importlib, identify_storms, pandas as pd
importlib.reload(identify_storms)

summary_df = identify_storms.identify_storms(
    input_dir="Rain_Data_High_Quality",
    output_dir="storms_identification",
    rain_col="rain",
    time_col="time",
    recursive=True
)
print("Statuses:\n", summary_df['status'].value_counts(), "\n")
print("Sample rows:\n", summary_df.head(10))


## Estimate rainfall intensity in each time interval

In [ ]:
import importlib, identify_storms, pandas as pd

importlib.reload(identify_storms)
import process_intervals
summary = process_intervals.process_directory(
    input_dir="storms_identification",   # your storm CSV tree
    output_dir="storms_intervals",       # where to save processed storms
    rain_col="rain",
    time_col="time",
    recursive=True
  
)
summary.head()



## Erosivity Storm

In [ ]:
import importlib, erosive_storms
importlib.reload(erosive_storms)

summary = erosive_storms.filter_erosive_storms(
    input_dir="storms_intervals",
    output_dir="erosive_storms",
    threshold_mm=12.7,  # 12.7 mm (~0.5 in)
    cumulative_col="cumulative rain depth (mm)",  # must match your files
    station_col="stid",
    time_col="time"
)
summary.head()


## Unite Rainfall Energy Estimation

In [ ]:
from importlib import reload
import rainfall_energy

reload(rainfall_energy)

summary = rainfall_energy.process_rainfall_energy(
    input_dir="erosive_storms",          # has subfolders per STID
    output_dir="storm_energy",  # mirrored structure with new columns
    intensity_col="rainfall intensity (mm/hr)",
    interval_mm_col="rain depth in interval (mm)",
    pattern="**/*.csv",                 # recursive
    overwrite=True
)
summary.head()

## Find Max Rainfall Intensity whinin 30 Minutes

In [ ]:
import importlib, max_30_min_rainfall
importlib.reload(max_30_min_rainfall)


written = max_30_min_rainfall.process_max_rolling(
    input_path="storm_energy",   
    output_dir = "max_rainfall",  # your folder
    recursive=True
)
written[:5]

## Estimate Erosivity for Individual Events

In [ ]:
import importlib, rainfall_erosivity
importlib.reload(rainfall_erosivity)

summary = rainfall_erosivity.process_rainfall_erosivity(
    input_dir="max_rainfall",        # folder with augmented CSVs (includes '30-min rainfall (mm)')
    output_dir="rainfall_erosivity",   # will save to <out>/<stid>/<stid>_<year>_<month>.csv
    recursive=True,
    write_combined=True
)
summary.head()

## Monthly Erosivity


In [ ]:
from importlib import reload
import monthly_erosivity

reload(monthly_erosivity)  # only needed if you edit the .py file
monthly_erosivity.process_monthly_erosivity(
    input_dir="rainfall_erosivity",
    output_dir="monthly_erosivity"
)
